# Combine model avg fits
Read fit results and combine them to give estimates for parameters

Dec 28, 2022

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import *
import glob 
import sys
import pickle 
import scipy.special as sc
# from scipy.special import logsumexp as sc.logsumexp

## Import fitting modules
import gvar as gv
import lsqfit


In [13]:
%matplotlib widget

In [15]:
from modules import *

### Load data for troubleshooting fits

In [16]:
if __name__=="__main__":
    
    ### Set values for 2pt function and speed of light #####
    dict_global={}
    dict_global['4']=  {'2pt':1.126500594310e-02, 'c':2.758810226094e-01}
    dict_global['8']=  {'2pt':5.591911340388e-03, 'c':1.395714549742e-01}
    dict_global['16']= {'2pt':2.790831203460e-03, 'c':6.999296068404e-02}
    print(dict_global)
    
    ### Load entire data ####
    data_dict={}

    s_list=[4,8,16]
    data_dir='../../data/free_theory/'

    for s in s_list:
        
        print(data_dir+'s2xr_free_q5k{0}t*_4pt_pl.dat'.format(s))
        fname=glob.glob(data_dir+'s2xr_free_q5k{0}t*_4pt_pl.dat'.format(s))[0]
        print(fname,s)
        df,Lt=f_get_data_df(fname,dict_global)
        data_dict[str(s)]={'df':df,'Lt':Lt}


{'4': {'2pt': 0.0112650059431, 'c': 0.2758810226094}, '8': {'2pt': 0.005591911340388, 'c': 0.1395714549742}, '16': {'2pt': 0.00279083120346, 'c': 0.06999296068404}}
../../data/free_theory/s2xr_free_q5k4t*_4pt_pl.dat
../../data/free_theory/s2xr_free_q5k4t64_4pt_pl.dat 4
4 64
../../data/free_theory/s2xr_free_q5k8t*_4pt_pl.dat
../../data/free_theory/s2xr_free_q5k8t128_4pt_pl.dat 8
8 128
../../data/free_theory/s2xr_free_q5k16t*_4pt_pl.dat
../../data/free_theory/s2xr_free_q5k16t256_4pt_pl.dat 16
16 256


### Read stored data and average

In [17]:
save_loc='../../data/stored_results/'

s_list=[4,8,16]

all_pars=["a{0}".format(i) for i in range(4)]+["E{0}".format(i) for i in range(4)]+['const']
cols=['s','l']+all_pars
df_results=pd.DataFrame(columns=cols)
    
    
for s in [str(i) for i in s_list][:1]:
    for l in [0,2,4,6][:2]:
        print(s,l)
        
#         Read fits from file
        fname=save_loc+'l{0}_s{1}.df_fits'.format(l,s)
        print(fname)
        df_fits=pd.read_pickle(fname)
        
        df_fits,df_avg=f_avg_all_models(df_fits)
        print(df_fits.shape)
        
        # Store result
        dict2=dict.fromkeys(all_pars,np.nan)
        dict2['s']=int(s)
        dict2['l']=l

        count=0
        pars=df_avg.columns
        for key in pars:    dict2[key]=df_avg.loc['value',key]
#         print(dict2)
        df_results=pd.concat([df_results,pd.DataFrame(dict2,index=[count])])
        count+=1

4 0
../../data/stored_results/l0_s4.df_fits
(6, 22)
4 2
../../data/stored_results/l2_s4.df_fits
(5, 22)


/Users/vayyar/Desktop/work/PROJECTS/summary_git_repo/Code_highlights/2_Correlated_Fits_QFE/fit_4pt_function/code/model_avg/modules.py:432: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fits['normed_prob']=df_fits.apply(lambda row : np.exp(row.fit_wt-S),axis=1)
/Users/vayyar/Desktop/work/PROJECTS/summary_git_repo/Code_highlights/2_Correlated_Fits_QFE/fit_4pt_function/code/model_avg/modules.py:432: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fits['normed_prob']=df_fits.apply(lambda row : np.exp(row.fi

### View results

In [18]:
df_results.sort_values(by=['l','s'])


,s,l,a0,a1,a2,a3,E0,E1,E2,E3,const
0,4,0,7.8559(17),2.72(54),3.1(4.2),NaN,0.999224(45),3.016(93),6.0(3.1),NaN,0.00231855(62)
0,4,2,0.000490(23),4.0068(48),1.47(39),NaN,1.9382(58),2.93787(30),4.68(17),NaN,7.91(28)e-12


## Troubleshooting 

### View fits dataframe

In [19]:
s=4;l=0
print(s,l)
fname=save_loc+'l{0}_s{1}.df_fits'.format(l,s)
df_fits=pd.read_pickle(fname)

4 0


In [11]:
sub_cols=['name','chi2_dof','fit_wt','num_exp','a0','a1','a2','a3','E0','E1','E2','E3']
df_fits[sub_cols].sort_values(by=['fit_wt'],ascending=False).head(15)

KeyError: "['a3', 'E3'] not in index"

In [18]:
# df_fits.sort_values(by=['chi2_dof'],ascending=True)

### Perform fit for specific trange

In [29]:
s='16';l=6
df_temp=data_dict[s]['df']
Lt=data_dict[s]['Lt']
df_temp=df_temp[df_temp.l==l][['t','coeff']]

Corr=corr(df=df_temp,l=l,c=dict_global[str(s)]['c'],s=s,Lt=Lt)

In [30]:
tmax=int(np.max(df_temp.t.values)) # Max value of t obtained from dataframe

func_w=Dropdown(options=[f_multi_exp,f_multi_exp2,f_cosh],value=f_multi_exp2,description='fit func',disabled=False)
tr=IntRangeSlider(value=[20,tmax],min=0,max=tmax,step=1,description='trange')
pr=IntRangeSlider(value=[0,tmax],min=0,max=tmax,step=1,description='plot range')
pr_w=widgets.Checkbox(value=False,description='use_prior',indent=False,disabled=False)
plt_w=Checkbox(value=True,description='plot',indent=False,disabled=False)
numexp_w=IntSlider(value=2,min=1,max=8,step=1,description='num_exp')
verbose_w=IntSlider(value=1,min=-1,max=2,step=1,description='verbose_w')

# (self,f_make_pars,func,fit_range,plt_range,num_exp=3,verbose=0,use_prior=False,plot=True)

v1=VBox([tr,pr])
v2=VBox([numexp_w,verbose_w])
v3=VBox([pr_w,plt_w])

ui=HBox(children=[v1,func_w,v2,v3])
        
out=interactive_output(Corr.f_perform_fit_exp, {'f_make_pars':fixed(f_make_pars),
         'func':func_w,'fit_range':tr, 'plt_range':pr,'num_exp':numexp_w, 
         'verbose':verbose_w,'use_prior':pr_w,'plot':plt_w})

display(out,ui)

Output()

In [ ]:
# Corr.f_perform_fit_exp(f_make_pars,f_multi_exp2,np.arange(6,24),[0,32],1,0,False,True) # l=0,1,2,3